# Configuration space
This notebook explores the configuration space of Elliptic Curve crypto
implementations.


An ECC implementation configuration in **pyecsca** has the following attributes: 

In [ ]:
from typing import get_args
from pyecsca.ec.configuration import Configuration
from dataclasses import fields

for field in fields(Configuration):
	name = field.name
	tp = field.type
	doc = tp.__doc__
	if get_args(field.type):
		doc = get_args(field.type)[0].__doc__
	print(name, tp)
	print("   ", doc)
	if hasattr(tp, "names"):
		for enum_name in tp.names():
			print("       ", enum_name)
	print()

It is represented by the `Configuration` class.

## Enumerating configurations

The possible configurations can be generated using the `all_configurations()` function.
The whole space of configurations is quite huge:

In [ ]:
from pyecsca.ec.configuration import all_configurations

print(sum(1 for _ in all_configurations()))

A large part of the configuration space is due to the independent options which consist of:
 
 - `hash_type` of type `HashType` $*6$
 - `mod_rand` of type `RandomMod` $*2$
 - `mult` of type `Multiplication` $*4$
 - `sqr` of type `Squaring` $*4$
 - `red` of type `Reduction` $*3$
 - `inv` of type `Inversion` $*2$

To restrict the generated configurations, pass keyword arguments to the
`all_configurations` matching the names of the attributes of the `Configuration` object.

In [ ]:
from pyecsca.ec.configuration import HashType, RandomMod, Multiplication, Squaring, Reduction, Inversion
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.mult import LTRMultiplier

model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
scalarmult = LTRMultiplier
independent_opts = {
	"hash_type": HashType.SHA256,
	"mod_rand": RandomMod.SAMPLE,
	"mult": Multiplication.KARATSUBA,
	"sqr": Squaring.KARATSUBA,
	"red": Reduction.MONTGOMERY,
    "inv": Inversion.GCD
}

configs = list(all_configurations(model=model, coords=coords, scalarmult=scalarmult,
							 	  **independent_opts))
print(len(configs))

We see that when we fixed all parameters except for the scalar multiplier arguments 
(see the `LTRMultiplier` constructor) we obtained 560 configurations.

### Models

In [ ]:
from IPython.display import HTML, display
import tabulate
from pyecsca.ec.model import *

model_counts = [["Model", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
totals = ["Total", 0, 0, 0, 0]
for model in (ShortWeierstrassModel(), MontgomeryModel(), EdwardsModel(), TwistedEdwardsModel()):
	name = model.__class__.__name__
	count = sum(1 for _ in all_configurations(model=model, **independent_opts))
	count_no_scl = sum(1 for _ in all_configurations(model=model, **independent_opts, scalarmult={"scl": None}))
	count_no_opts = sum(1 for _ in all_configurations(model=model, **independent_opts, scalarmult={"scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3}))
	model_counts.append([name, count * (6*2*4*4*3*2), count, count_no_scl, count_no_opts])
	totals[1] += count * (6*2*4*4*3*2)
	totals[2] += count
	totals[3] += count_no_scl
	totals[4] += count_no_opts
model_counts.append(totals)
display(HTML(tabulate.tabulate(model_counts, tablefmt="html", headers="firstrow")))

### Coordinate systems

In [ ]:
coords_counts = [["Model", "Coords", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
for model in (ShortWeierstrassModel(), MontgomeryModel(), EdwardsModel(), TwistedEdwardsModel()):
	model_name = model.__class__.__name__
	coords_counts.append([model_name, "", "", "", "", ""])
	for coords in sorted(model.coordinates.values(), key=lambda c: c.name):
		coords_name = coords.name
		count = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts))
		count_no_scl = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts, scalarmult={"scl": None}))
		count_no_opts = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts, scalarmult={"scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3}))
		coords_counts.append(["", coords_name, count * (6*2*4*4*3*2), count, count_no_scl, count_no_opts])
display(HTML(tabulate.tabulate(coords_counts, tablefmt="html", headers="firstrow")))

### Scalar multipliers

In [ ]:
from pyecsca.ec.mult import ScalarMultiplier

mult_counts = [["ScalarMultiplier", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
for mult_cls in ScalarMultiplier.__subclasses__():
	count = sum(1 for _ in all_configurations(**independent_opts, scalarmult=mult_cls))
	count_no_scl = sum(1 for _ in all_configurations(**independent_opts, scalarmult={"cls": mult_cls, "scl": None}))
	count_no_opts = sum(1 for _ in all_configurations(**independent_opts, scalarmult={"cls": mult_cls, "scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3}))
	mult_counts.append([mult_cls.__name__, count * (6*2*4*4*3*2), count, count_no_scl, count_no_opts])
display(HTML(tabulate.tabulate(mult_counts, tablefmt="html", headers="firstrow")))
